<a href="https://colab.research.google.com/github/Chinmaya-3141/Machine-Learning-Emission-Data/blob/edit1/25_01_2023_MSE_Presentation_File.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Gas Turbine Emission Data**

### Fetch Data

In [1]:
from datetime import datetime
import pytz
print('Code execution last started at  ',datetime.now(pytz.timezone('Asia/Kolkata')))

Code execution last started at   2023-02-13 16:00:53.647251+05:30


In [ ]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/00551/pp_gas_emission.zip

--2023-02-13 10:30:53--  https://archive.ics.uci.edu/ml/machine-learning-databases/00551/pp_gas_emission.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.


In [ ]:
# !gdown 1otUkltGJRfbVDTvCmElLTnmnV-4FEtrF

In [ ]:
!unzip pp_gas_emission.zip

## Preprocessing

### Import Data

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
data_2015=pd.concat([pd.read_csv("gt_2011.csv"),\
                     pd.read_csv("gt_2012.csv"),\
                     pd.read_csv("gt_2013.csv"),\
                     pd.read_csv("gt_2014.csv"),\
                     pd.read_csv("gt_2015.csv")],\
                    axis=0)

In [ ]:
data_2015.shape

In [ ]:
data_2015.describe()

### EDA

In [ ]:
# Showing correlation of all variables among each other and not selecting target variables

plt.figure(figsize=(12,12))
sns.heatmap(data_2015.loc[:,'AT':'CDP'].corr(),annot=True)
# plt.savefig("corr.png")

In [ ]:
# import seaborn as sns
# import matplotlib.pyplot as plt
# plt.rcParams['figure.figsize']=(16,7)
# f, (ax1,ax2,ax3,ax4,ax5,ax6,ax7,ax8,ax9) = plt.subplots(1,9)
# sns.boxplot(data=data_2015['AT'], ax=ax1, palette = 'coolwarm')
# sns.boxplot(data=data_2015['AP'], ax=ax2)
# sns.boxplot(data=data_2015['AH'], ax=ax3, palette = 'coolwarm')
# sns.boxplot(data=data_2015['AFDP'], ax=ax4)
# sns.boxplot(data=data_2015['GTEP'], ax=ax5, palette='coolwarm')
# sns.boxplot(data=data_2015['TIT'], ax=ax6)
# sns.boxplot(data=data_2015['TAT'], ax=ax7, palette='coolwarm')
# sns.boxplot(data=data_2015['TEY'], ax=ax8)
# sns.boxplot(data=data_2015['CDP'], ax=ax9, palette = 'coolwarm')
# f.tight_layout()
# plt.savefig("boxplot.png")

In [ ]:
# data_2015.to_csv()

### Outlier Treatment

In [ ]:
#LOF is a robust outlier detection method in ML

from sklearn.neighbors import LocalOutlierFactor
from sklearn.preprocessing import StandardScaler

In [ ]:
# Scaling all variables except for target variables

data_2015_scaled= StandardScaler().fit_transform(data_2015.loc[:,'AT':'CDP'])

In [ ]:
data_2015.shape

In [ ]:
# Using Local Outlier Factor to identify potential outliers

lof=LocalOutlierFactor(n_neighbors=5)
pred = lof.fit_predict(data_2015_scaled)

In [ ]:
pd.Series(pred).value_counts()

In [ ]:
# initialising an empty array, passing no of rows/entries using shape[0]
# storing predictions for 5,7,9 neighbours in all_preds by looping n_neigh over range(5,10,2)

all_preds = np.zeros(shape=(data_2015_scaled.shape[0],3))
i=0
for n_neigh in range(5,10,2):
  lof=LocalOutlierFactor(n_neighbors=n_neigh)
  pred=lof.fit_predict(data_2015_scaled)
  all_preds[:,i]=pred
  i+=1

In [ ]:
all_preds

In [ ]:
# Sum along rows, that is sum prediction whether outlier or not according to 5,7 and 9 neighbour prediction

outlier_preds = all_preds.sum(axis=1)

In [ ]:
# outlier_preds

In [ ]:
# Summed values

# np.unique(outlier_preds)

In [ ]:
# Pick points which are considered outliers by at least two out of three n_neighbours lof
# This will happen when sum of row is 1 or 3
# final_outlier assigns 1 to outliers
# This is done by checking if rowsum is >= 1, that is 1 or 3
# If two lof numbers say not outlier, we keep it
# Like a voting method.

final_outlier=np.where(outlier_preds>=1,1,-1)

In [ ]:
final_outlier

In [ ]:
# Checking percentage of Outliers to non-outliers from our prediction

pd.Series(final_outlier).value_counts(normalize=True)

Consensus outliers:

(963/36733)*100 \
= 2.6216208858519587%

In [ ]:
data_2015_no_outlier = data_2015[final_outlier==1]

## Multiple Linear Regression

### Import Modules

In [ ]:
import statsmodels.formula.api as sm

In [ ]:
columns=data_2015.columns

In [ ]:
columns

In [ ]:
# data_2015_NOx=data_2015_no_outlier.drop('CO',axis=1)
# data_2015_CO=data_2015_no_outlier.drop('NOX',axis=1)

### Linear Regression

In [ ]:
# Join column numbers 1 to 9 to pass string with formula for CO

f1='CO~'+'+'.join(columns[:9])
f1

In [ ]:
r_co=sm.ols(formula=f1, data=data_2015_no_outlier)

In [ ]:
result_co=r_co.fit()

In [ ]:
result_co.summary()

In [ ]:
# Join column numbers 1 to 9 to pass string with formula for NOx

f2='NOX~'+'+'.join(columns[:9])
f2

In [ ]:
# OLS Regression

r_nox=sm.ols(formula=f2, data=data_2015_no_outlier)

In [ ]:
result_nox=r_nox.fit()

In [ ]:
result_nox.summary()

### Variable selection using Variance Inflation Factor (VIF)

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [ ]:
variance_inflation_factor(data_2015_no_outlier.loc[:,'AT':'CDP'].values,0)

In [ ]:
[i for i in range(10)]

In [ ]:
X_1 = data_2015_no_outlier.loc[:,'AT':'CDP'].copy()

In [ ]:
vif_scores = {k: v for k, v in zip(X_1.columns,
                                   [variance_inflation_factor(X_1.values, i) \
                                    for i in range(len(X_1.columns))])}

In [ ]:
vif_scores_sorted = {k:v for k, v in sorted(vif_scores.items(),key=lambda item:item[1], reverse = True)}

In [ ]:
vif_scores_sorted

In [ ]:
# X_1.columns.drop('TIT')

#### Defining functions to select variables with lowest VIF

In [ ]:
def var_selection_vif(data_exog: pd.DataFrame, th: int=5):
  vif_scores = {k: v for k, v in zip(data_exog.columns,
                                     [variance_inflation_factor(data_exog.values,i)\
                                      for i in range(len(data_exog.columns))])}
  vif_scores_sorted = {k:v for k, v in sorted(vif_scores.items(),
                                            key=lambda item:item[1],
                                            reverse = True)}
  if vif_scores_sorted[list(vif_scores_sorted.keys())[0]]>th:
    vars = data_exog.columns.drop(list(vif_scores_sorted.keys())[0])
    return (data_exog[vars], vif_scores_sorted)
  else:
    return(data_exog, vif_scores_sorted)

In [ ]:
n_old = len(X_1.columns)
n_new = 0
while n_old != n_new:
  X_1, vif = var_selection_vif(X_1)
  print(vif)
  n = len(X_1.columns)
  n_old = n_new
  n_new= n

### Regression with variables having appropriate VIF

In [ ]:
r_final_co = sm.ols(formula='CO~AT+AH', 
                 data = data_2015_no_outlier[list(X_1.columns)+['CO']])
result_final_co = r_final_co.fit()
print(result_final_co.summary())

In [ ]:
r_final_nox = sm.ols(formula='NOX~AT+AH', 
                 data = data_2015_no_outlier[list(X_1.columns)+['NOX']])
result_final_nox = r_final_nox.fit()
print(result_final_nox.summary())

### Variable selection using Lasso regression coefficients

In [ ]:
from sklearn.linear_model import Lasso

In [ ]:
lasso_reg1 = Lasso(alpha=2)
lasso_reg1.fit(X=data_2015_no_outlier.loc[:,'AT':'CDP'],\
              y=data_2015_no_outlier['CO'])

In [ ]:
CO_vars = {k:v for k,v in zip(data_2015_no_outlier.loc[:,'AT':'CDP'].columns, lasso_reg1.coef_)}

In [ ]:
lasso_reg1.score(X=data_2015_no_outlier.loc[:,'AT':'CDP'],\
              y=data_2015_no_outlier['CO'])

In [ ]:
lasso_reg2 = Lasso(alpha=2)
lasso_reg2.fit(X=data_2015_no_outlier.loc[:,'AT':'CDP'],\
               y=data_2015_no_outlier['NOX'])

In [ ]:
NOx_vars = {k:v for k,v in zip(data_2015_no_outlier.loc[:,'AT':'CDP'].columns, lasso_reg2.coef_)}

In [ ]:
lasso_reg2.score(X=data_2015_no_outlier.loc[:,'AT':'CDP'],\
                 y=data_2015_no_outlier['NOX'])

### Analysis of errors

In [ ]:
errors1 = data_2015_no_outlier['CO'] - lasso_reg1.predict(data_2015_no_outlier.loc[:,'AT':'CDP'])
d1=sns.displot(errors1)
plt.savefig('error-plot-CO.png')

In [ ]:
errors2 = data_2015_no_outlier['NOX'] - lasso_reg2.predict(data_2015_no_outlier.loc[:,'AT':'CDP'])
d2=sns.displot(errors2)
plt.savefig('error-plot-NOx.png')

In [ ]:
import statsmodels.api as smf

In [ ]:
smf.qqplot(errors1)
plt.show()
plt.savefig('error-dist-CO.png')

In [ ]:
smf.qqplot(errors2)
plt.show()
plt.savefig('error-dist-NOx.png')

### Multiple Linear Regression with relevant variables

In [ ]:
r_co_final_combined = sm.ols(formula='CO~AT+AH+TIT+TAT',\
                             data = data_2015[['AT','AP','AH','TIT','TAT']+['CO']])
result_co = r_co_final_combined.fit()
print(result_co.summary())

In [ ]:
r_nox_final_combined = sm.ols(formula='NOX~AT+AP+AH+TIT+TAT+TEY',\
                              data = data_2015[['AT','AP','AH','TIT','TAT','TEY']+['NOX']])
result_nox = r_nox_final_combined.fit()
print(result_nox.summary())

### Cook's Distance

In [ ]:
#cook's distnace tells the change in model when a single datapoint is dropped

In [ ]:
from statsmodels.stats.outliers_influence import OLSInfluence as influence

In [ ]:
inf1 = influence(result_co)

In [ ]:
inf2 = influence(result_nox)

In [ ]:
inf1.cooks_distance

In [ ]:
inf2.cooks_distance

In [ ]:
# Something needs to be changed here

inf1.summary_frame()[inf1.summary_frame()['cooks_d']>=4/7384]

In [ ]:
# Something needs to be changed here

inf2.summary_frame()[inf2.summary_frame()['cooks_d']>=4/7384]

## Machine Learning models

### Importing Relevant modules

In [ ]:
# !pip install xgboost
# !pip install lightgbm

In [ ]:
from hyperopt import fmin, hp, STATUS_OK, tpe , Trials
from hyperopt.pyll.base import scope
import hyperopt

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor, RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

In [ ]:
lin_reg = LinearRegression()
dt_reg = DecisionTreeRegressor()
rf_reg = RandomForestRegressor()
gbm_reg = GradientBoostingRegressor()
xgb_reg = XGBRegressor()
lgbm_reg = LGBMRegressor()

### Setting up parameter search space for hyperopt

In [ ]:
gt_new = data_2015.loc[:,'AT':'CO']
gt_new.shape

In [ ]:
param_dt={
    'max_depth': scope.int(hp.quniform('max_depth',2,20,1)),
    'ccp_alpha': hp.uniform('ccp_alpha',0.001,0.1)
}

param_rf={
    'n_estimators':scope.int(hp.quniform('n_estimators',50,500,1)),
    'max_features':hp.choice('max_features',list(range(2,7)))
}

param_gbm = {
    'max_depth':scope.int(hp.quniform('max_depth',1,6,1)),
    'n_estimators':scope.int(hp.quniform('n_estimators',50,500,1)),
    'learning_rate':hp.uniform('learning_rate',0.001,0.1)
}

param_xgb = {
    'max_depth':scope.int(hp.quniform('max_depth',1,6,1)),
    'n_estimators':scope.int(hp.quniform('n_estimators',50,500,1)),
    'learning_rate':hp.uniform('learning_rate',0.001,0.1),
    'colsample_bytree':hp.uniform('colsample_bytree',0.2,0.8)
}

param_lgbm = {
    # 'boosting_type': 'rf',
    'max_depth': scope.int(hp.quniform('max_depth',1,7,1)),
    'learning_rate': hp.uniform('learning_rate',0.001,0.1),
    'n_estimators': scope.int(hp.quniform('n_estimators',50,500,1)),
    'colsample_bytree': hp.uniform('colsample_bytree',0.2,0.8)
}

In [ ]:
from sklearn.metrics import make_scorer, mean_squared_error
from sklearn.model_selection import train_test_split, cross_val_score

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(gt_new.loc[:, 'AT':'CDP'],gt_new['CO'],test_size=0.3, random_state=123)

In [ ]:
def rmse(y_act, y_pred):
  return (np.sqrt(np.mean((y_act-y_pred)**2)))

rmse = make_scorer(rmse, greater_is_better=False)

In [ ]:
def hp_obj_func_dt(param):
  dt= DecisionTreeRegressor(**param)
  out=cross_val_score(dt, X=x_train, y= y_train, scoring='neg_root_mean_squared_error', cv=5).mean()
  print(out)
  return out

In [ ]:
def surr_func_dt(params):
  perf= hp_obj_func_dt(params)
  return {'loss':-perf,'status': STATUS_OK}

### Linear Regression

In [ ]:
lin_reg.fit(x_train, y_train)
pred_lin_reg = lin_reg.predict(x_test)
print(mean_squared_error(pred_lin_reg, y_test))

### Decision Tree

In [ ]:
trials = Trials()
best_dt = fmin(surr_func_dt, space=param_dt, algo=tpe.suggest,
               max_evals=100, trials=trials)
print('best_dt:')
print(best_dt)

In [ ]:
dt_reg = DecisionTreeRegressor(**best_dt)
dt_reg.fit(x_train, y_train)
pred_dt = dt_reg.predict(x_test)
print(mean_squared_error(pred_dt, y_test))

### Random forest




In [ ]:
def hp_obj_func_rf(param):
  rf= RandomForestRegressor(**param)
  out=cross_val_score(rf, X=x_train, y= y_train, scoring='neg_root_mean_squared_error', cv=5).mean()
  print(out)
  return out

In [ ]:
def surr_func_rf(params):
  perf= hp_obj_func_rf(params)
  return {'loss':-perf,'status': STATUS_OK}

In [ ]:
trials = Trials()
best_rf = fmin(surr_func_rf, space=param_rf, algo=tpe.suggest,
               max_evals=25, trials=trials)
print('best_rf:')
print(best_rf)

In [ ]:
# best_rf={'max_features': 2, 'n_estimators': 451}

In [ ]:
rf_reg = RandomForestRegressor(**best_rf)
rf_reg.fit(x_train, y_train)
pred_rf = rf_reg.predict(x_test)
print(mean_squared_error(pred_rf, y_test))

In [ ]:
plt.figure(figsize=(12,12))
for i in range(1,10):
  plt.subplot(3,3,i)
  sns.distplot(data_2015.iloc[:i-1])

### GBM

In [ ]:
def hp_obj_func_gbm(param):
  gbm= GradientBoostingRegressor(**param)
  out=cross_val_score(gbm, X=x_train, y= y_train, scoring='neg_root_mean_squared_error', cv=5).mean()
  print(out)
  return out

In [ ]:
def surr_func_gbm(params):
  perf= hp_obj_func_gbm(params)
  return {'loss':-perf,'status': STATUS_OK}

In [ ]:
trials = Trials()
best_gbm = fmin(surr_func_gbm, space=param_gbm, algo=tpe.suggest,
               max_evals=100, trials=trials)
print('best_gbm:')
print(best_gbm)

In [ ]:
best_gbm

In [ ]:
gbm_reg = GradientBoostingRegressor(**best_gbm)
gbm_reg.fit(x_train, y_train)
pred_gbm = gbm_reg.predict(x_test)
print(mean_squared_error(pred_gbm, y_test))

In [ ]:
gbm_reg = GradientBoostingRegressor(**best_gbm)
gbm_reg.fit(x_train, y_train)
pred_gbm = gbm_reg.predict(x_test)
print(mean_squared_error(pred_gbm, y_test))

### XgBoost

In [ ]:
def hp_obj_func_xgb(param):
  xgb= XGBRegressor(**param)
  out=cross_val_score(xgb, X=x_train, y= y_train, scoring='neg_root_mean_squared_error', cv=5).mean()
  print(out)
  return out

In [ ]:
def surr_func_xgb(params):
  perf= hp_obj_func_xgb(params)
  return {'loss':-perf,'status': STATUS_OK}

In [ ]:
trials = Trials()
best_xgb = fmin(surr_func_xgb, space=param_xgb, algo=tpe.suggest,
               max_evals=100, trials=trials)
print('best_xgb:')
print(best_xgb)

In [ ]:
# best_xgb = {'colsample_bytree': 0.5904951775918889, 'learning_rate': 0.024729072063200287, 'max_depth': 5, 'n_estimators': 267}

In [ ]:
# best_xgb = {'colsample_bytree': 0.7974445913518547, 'learning_rate': 0.09830911482175203, 'max_depth': 5, 'n_estimators': 380}

In [ ]:
best_xgb

In [ ]:
xgb_reg = XGBRegressor(**best_xgb)
xgb_reg.fit(x_train, y_train)
pred_xgb = xgb_reg.predict(x_test)
print(mean_squared_error(pred_xgb, y_test))

In [ ]:
xgb_reg = XGBRegressor(**best_xgb)
xgb_reg.fit(x_train, y_train)
pred_xgb = xgb_reg.predict(x_test)
print(mean_squared_error(pred_xgb, y_test))

In [ ]:
plt.figure(figsize=(12,12))
for i in range(1,10):
  plt.subplot(3,3,i)
  sns.distplot(data_2015.iloc[:i-1])

### LightGBM

In [ ]:
def hp_obj_func_lgbm(param):
  lgbm= LGBMRegressor(**param)
  out=cross_val_score(lgbm, X=x_train, y= y_train, scoring='neg_root_mean_squared_error', cv=5).mean()
  print(out)
  return out

In [ ]:
def surr_func_lgbm(params):
  perf= hp_obj_func_lgbm(params)
  return {'loss':-perf,'status': STATUS_OK}

In [ ]:
trials = Trials()
best_lgbm = fmin(surr_func_lgbm, space=param_lgbm, algo=tpe.suggest,
               max_evals=500, trials=trials)
print('best_lgbm:')
print(best_lgbm)

In [ ]:
# best_lgbm = {'colsample_bytree': 0.6669160688292152,
#              'learning_rate': 0.07968656287632815,
#              'max_depth': 6,
#              'n_estimators': 491}

In [ ]:
lgbm_reg = LGBMRegressor( **best_lgbm)
lgbm_reg.fit(x_train, y_train)
pred_lgbm = lgbm_reg.predict(x_test)
print(mean_squared_error(pred_lgbm, y_test))

In [ ]:
# lgbm = LGBMRegressor(boosting_type = 'rf', \
#                      num_leaves = \
#                      max_depth = \
#                      learning_rate = \
#                      n_estimators = \
#                      objective='regression', \
#                      random_state=123\
#                      n_jobs)

# lgbm.fit(X, y)

# y_pred = lgbm.predict(X_test)

### SVM

In [ ]:
from sklearn.svm import SVR
svr_linear = SVR(kernel='linear',gamma='scale', C=1.0, epsilon=0.1)
svr_linear.fit(x_train, y_train)

In [ ]:
svr_linear.score(x_test,y_test)

In [ ]:
svr_rbf = SVR(kernel='rbf',gamma='scale', C=1.0, epsilon=0.1)
svr_rbf.fit(x_train, y_train) 

In [ ]:
svr_rbf.score(x_test,y_test)

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
print("RMSE for linear SVR:",np.sqrt(mean_squared_error(y_test,svr_linear.predict(x_test))))
print("RMSE for RBF kernelized SVR:",np.sqrt(mean_squared_error(y_test,svr_rbf.predict(x_test))))

### Code Last Run:

In [ ]:
from datetime import datetime
import pytz
print('signed off at  ',datetime.now(pytz.timezone('Asia/Kolkata')))

## Hypothesis Testing

###Z-Test 2 sample test


In [ ]:
import math
import numpy as np
from numpy.random import randn
from statsmodels.stats.weightstats import ztest as ztest
import pandas as pd



In [ ]:
df1=pd.concat([pd.read_csv("gt_2011.csv"),pd.read_csv("gt_2012.csv"),pd.read_csv("gt_2013.csv"),pd.read_csv("gt_2014.csv")],axis=0)
df2=pd.read_csv("gt_2015.csv")
df3=pd.read_csv("gt_2014.csv")


In [ ]:
ztest(df3['NOX'], value=np.mean(df2['NOX']))

Conclusion: The mean value of NOX emission for 2015 is  equal to the mean value of the NOX emissions of years from 2014.

In [ ]:
ztest(df3['CO'], value= np.mean(df2['CO'])) 

Conclusion: The mean value of CO emission for 2015 is not equal to the mean value of the CO emissions of years from 2014.

###ANOVA

In [ ]:
from scipy.stats import f_oneway
# result= f_oneway(pd.read_csv("gt_2011.csv")['NOX'],pd.read_csv("gt_2012.csv")['NOX'],pd.read_csv("gt_2013.csv")['NOX'],pd.read_csv("gt_2014.csv")['NOX'],df2['NOX'])
result= f_oneway(pd.read_csv("gt_2011.csv")['NOX'],pd.read_csv("gt_2012.csv")['NOX'])
result